## End to end model:Classifier

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.applications.vgg19 import preprocess_input
from keras.utils.training_utils import multi_gpu_model
from keras.utils import to_categorical
import tensorflow as tf
import cv2
import os
import random
import h5py
import numpy as np

Using TensorFlow backend.


## 1 read feature and label

In [2]:
h5_files = ['/home/tsimage/high_speed_data/end2end_feature/feature_label_500.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_1000.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_1500.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_2000.h5']

for i, h5_file in enumerate(h5_files):
    h5f = h5py.File(h5_file,'r')
    features_h5 = h5f['features'][:]
    labels_h5 = h5f['labels'][:]
    h5f.close()
    if (i == 0):
        print("du ###",i)
        features_h5_all = features_h5
        labels_h5_all = labels_h5
        print("du", features_h5_all.shape)
        print("du", labels_h5_all.shape)
        print("du", features_h5.shape)
        print("du", labels_h5.shape)
        features_h5 = None
        labels_h5 = None
    else:
        print("du ***",i)
        print("du", features_h5_all.shape)
        print("du", labels_h5_all.shape)
        print("du", features_h5.shape)
        print("du", labels_h5.shape)
        features_h5_all = np.append(features_h5_all, features_h5, axis = 0)
        labels_h5_all = np.append(labels_h5_all, labels_h5,  axis = 0)
        features_h5 = None
        labels_h5 = None
        
print("du", features_h5_all.shape)
print("du", labels_h5_all.shape)

du ### 0
du (500, 128, 128, 512)
du (500,)
du (500, 128, 128, 512)
du (500,)
du *** 1
du (500, 128, 128, 512)
du (500,)
du (491, 128, 128, 512)
du (491,)
du *** 2
du (991, 128, 128, 512)
du (991,)
du (500, 128, 128, 512)
du (500,)
du *** 3
du (1491, 128, 128, 512)
du (1491,)
du (500, 128, 128, 512)
du (500,)
du (1991, 128, 128, 512)
du (1991,)


In [3]:
features = features_h5_all
labels = labels_h5_all.reshape([len(labels_h5_all),1])
features_h5_all = None
labels_h5_all = None

print("du", features.shape)
print("du", labels.shape)

labels = to_categorical(labels,2)
print("du", features.shape)
print("du", labels.shape)

index = [i for i in range(len(features))]  
random.shuffle(index)
features = features[index]
labels = labels[index]

print("du", features.shape)
print("du", labels.shape)

du (1991, 128, 128, 512)
du (1991, 1)
du (1991, 128, 128, 512)
du (1991, 2)
du (1991, 128, 128, 512)
du (1991, 2)


## 2 build classifier

In [4]:
with tf.device('/cpu:0'):
    input_tensor = Input((128, 128, 512))
    
    conv1 = Conv2D(filters = 1024, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv1')(input_tensor)
    conv2 = Conv2D(filters = 512, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv2')(conv1)
    
    conv3 = Conv2D(filters = 1024, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv3')(conv2)
    conv4 = Conv2D(filters = 512, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv4')(conv3)
    
    conv5 = Conv2D(filters = 1024, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv5')(conv4)
    conv6 = Conv2D(filters = 512, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv6')(conv5)
    
    conv7 = Conv2D(filters = 1024, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv7')(conv6)
    conv8 = Conv2D(filters = 512, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv8')(conv7)

#    flatten = Flatten(name='flatten')(conv8)
#    fc1 = Dense(1024, activation='relu', name='fc1')(flatten)
#    gap = GlobalAveragePooling2D()(conv8)
    gmp = GlobalMaxPooling2D()(conv8)
    drop = Dropout(0.5, name='drop_fc1')(gmp)
    predictions = Dense(2, activation='softmax')(drop)

    model = Model(inputs=input_tensor, outputs=predictions)
    model.summary()

    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

parallel_model = multi_gpu_model(model, gpus=4)
#parallel_model.compile(optimizer='Adadelta', 
#                       loss='categorical_crossentropy', 
#                       metrics=['accuracy'])
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
parallel_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 512)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 1024)      4719616   
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 64, 512)       524800    
_________________________________________________________________
conv3 (Conv2D)               (None, 32, 32, 1024)      4719616   
_________________________________________________________________
conv4 (Conv2D)               (None, 32, 32, 512)       524800    
_________________________________________________________________
conv5 (Conv2D)               (None, 16, 16, 1024)      4719616   
_________________________________________________________________
conv6 (Conv2D)               (None, 16, 16, 512)       524800    
__________

In [6]:
print("du", features.shape)
print("du", labels.shape)
parallel_model.fit(x=features, y=labels, 
                   batch_size=16, epochs=1, verbose=1, validation_split=0.2)
model.save_weights("diagnosis.h5")

du (1991, 128, 128, 512)
du (1991, 2)
Train on 1592 samples, validate on 399 samples
Epoch 1/1
1592/1592 [==============================] - 86s 54ms/step - loss: 0.0622 - acc: 0.9793 - val_loss: 0.5109 - val_acc: 0.8672
